In [6]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import pandas as pd
import os
import glob

In [7]:
models_list = ['MLP', 'RWF', 'KAN']

In [9]:
u_min, u_max = -0.21, 0.0
x_min, x_max = 0.0, 1.0
y_min, y_max = 0.0, 1.0
t_f = 10
f_min, f_max = -3.0, 0.0

def hard_constraint(x_in, U_theta):
    delta_u = u_max - u_min
    delta_x = x_max - x_min
    delta_y = y_max - y_min

    X = x_in[0]
    Y = x_in[1]
    tau = x_in[-1]
    x = X*delta_x + x_min
    y = Y*delta_y + y_min
    t = tau * t_f
    u_theta = U_theta*delta_u + u_min

    # se siamo sui bordi (qualsiasi dimensione) u=0, altrimenti abbiamo applicato una trasformazione non lineare a u_theta
    u = u_theta * (x-x_min) *(x-x_max) * (y-y_min) * (y-y_max) * t
    U = (u - u_min)/delta_u # forma esplicita: riga 72 in 73, poi righe 66-69 

    # output normalizzato
    return U

In [10]:
# Paths and setup
csv_path = 'matlab/only_damp/space_time_points.csv'
batch_size = 500

df = pd.read_csv(csv_path)

# Convert pandas Series to numpy arrays
X = df['x'].to_numpy()
Y = df['y'].to_numpy()
T = df['t'].to_numpy()

n_samples = len(X)
print(n_samples)

predictions = np.zeros(n_samples)

# Prepare input data as tensor
inputs = np.column_stack((X, Y, T))
inputs = torch.FloatTensor(inputs)


191700


In [ ]:
for model_name in models_list:
    for i in range(1, 7):
        model_path = f"training/{model_name}_{i}/model"

        output_path = f'matlab/only_damp/matlab_evaluations/predictions_csv_files/{model_name}_{i}'
        if not os.path.exists(output_path):
            os.makedirs(output_path)
        
        model_files = glob.glob(os.path.join(model_path, "model_*.pt"))
        if model_files:
        # Extract numbers and find the highest one
            latest_model = max(model_files, key=lambda x: int(x.split("_")[-1].split(".")[0]))
            print("Evaluating model in:", latest_model)
        else:
            print("No model files found in the directory.")

        #model_path = latest_model

        loaded_model = torch.load(latest_model)
        loaded_model.eval()
        
        # Process in batches
        with torch.no_grad():
            for i in range(0, n_samples):
                # end_idx = min(i + batch_size, n_samples)  # Ensure we don't go past the array end
                batch_inputs = inputs[i]
                batch_outputs = loaded_model(batch_inputs)
                pred = batch_outputs.cpu().numpy()
                pred_denorm = (pred*(u_max - u_min)) + u_min
                predictions[i] = pred_denorm # Only take the needed values
            
                print(pred_denorm)

        # Formatting predictions for matlab use
        reshaped_predictions = predictions.reshape(-1, 1917).T
        np.savetxt(output_path + '/predictions.csv', reshaped_predictions, 
           delimiter=',', fmt='%.6f')  

        print(f"Predictions completed and saved to {output_path}")
        print("\nSample predictions:")
        print(df.head())

        # Print statistics about the predictions
        print("\nPrediction statistics:")
        print(f"Mean: {predictions.mean():.4f}")
        print(f"Std: {predictions.std():.4f}")
        print(f"Min: {predictions.min():.4f}")
        print(f"Max: {predictions.max():.4f}")

In [ ]:
model_path = f"training/KAN_7/model"

output_path = f'matlab/only_damp/matlab_evaluations/predictions_csv_files/KAN_7'
if not os.path.exists(output_path):
    os.makedirs(output_path)

model_files = glob.glob(os.path.join(model_path, "model_*.pt"))
if model_files:
# Extract numbers and find the highest one
    latest_model = max(model_files, key=lambda x: int(x.split("_")[-1].split(".")[0]))
    print("Evaluating model in:", latest_model)
else:
    print("No model files found in the directory.")

#model_path = latest_model

loaded_model = torch.load(latest_model)
loaded_model.eval()

# Process in batches
with torch.no_grad():
    for i in range(0, n_samples):
        # end_idx = min(i + batch_size, n_samples)  # Ensure we don't go past the array end
        batch_inputs = inputs[i]
        batch_outputs = loaded_model(batch_inputs)
        pred = batch_outputs.cpu().numpy()
        pred_denorm = (pred*(u_max - u_min)) + u_min
        predictions[i] = pred_denorm # Only take the needed values
    
        print(pred_denorm)

# Formatting predictions for matlab use
reshaped_predictions = predictions.reshape(-1, 1917).T
np.savetxt(output_path + '/predictions.csv', reshaped_predictions, 
    delimiter=',', fmt='%.6f')  

print(f"Predictions completed and saved to {output_path}")
print("\nSample predictions:")
print(df.head())

# Print statistics about the predictions
print("\nPrediction statistics:")
print(f"Mean: {predictions.mean():.4f}")
print(f"Std: {predictions.std():.4f}")
print(f"Min: {predictions.min():.4f}")
print(f"Max: {predictions.max():.4f}")